# Code Conversion using Ollama Inference Models

In [1]:
from itertools import permutations
from constants import *

import ollama
import os
import re

In [2]:
RESULTS_FOLDER = "results"
GENERATED_CODE_FOLDER = os.path.join(RESULTS_FOLDER, "generated_code")
REJECTED_CODE_FOLDER = os.path.join(RESULTS_FOLDER, "rejected_code")
MODEL = "dolphincoder7b"
LANG2LANG = list(permutations(LANGS.keys(), 2))

In [3]:
def get_prompt(system_prompt, user_prompt):
    prompt = []
    if system_prompt:
        prompt.append({"role": "system", "content": system_prompt})
    prompt.append({"role": "user", "content": user_prompt})
    return prompt

In [4]:
prompt = "Convert the following code from {} to {}. This is the requirement for the code - {}\n```{}```\n"

In [5]:
ollama.pull(OLLAMA_TAGS[MODEL])

{'status': 'success'}

In [6]:
for TASK in TASKS.keys():
    for idx, (lang1, lang2) in enumerate(LANG2LANG):
        if REFERENCE_CODE[lang1][TASK] == "":
            continue
        if os.path.exists(
            os.path.join(
                GENERATED_CODE_FOLDER,
                f"{MODEL}-{TASK}-{lang1.lower()}-{lang2.lower()}.md",
            )
        ):
            continue
        if os.path.exists(
            os.path.join(
                REJECTED_CODE_FOLDER,
                f"{MODEL}-{TASK}-{lang1.lower()}-{lang2.lower()}.md",
            )
        ):
            continue
        formatted_prompt = prompt.format(
            LANGS[lang1],
            LANGS[lang2],
            TASK_DESCRIPTION[TASK],
            REFERENCE_CODE[lang1][TASK],
        )
        if OLLAMA_SYSTEM_PROMPT_SUPPORT[MODEL]:
            chat_prompt = get_prompt(
                "You are a helpful code conversion assistant.", formatted_prompt
            )
        else:
            chat_prompt = get_prompt(None, formatted_prompt)
        try:
            rsp = ollama.chat(
                OLLAMA_TAGS[MODEL],
                messages=chat_prompt,
                options={"seed": 42, "timeout": 60},
            )
        except:
            try:
                rsp = ollama.chat(
                    OLLAMA_TAGS[MODEL],
                    messages=chat_prompt,
                    options={"seed": 41, "timeout": 60},
                )
            except:
                print(f"timeout in {MODEL} {TASK} {lang1} {lang2}")
                with open(
                    os.path.join(
                        REJECTED_CODE_FOLDER,
                        f"{MODEL}-{TASK}-{lang1.lower()}-{lang2.lower()}.md",
                    ),
                    "w",
                ) as fp:
                    fp.write("\n")
                continue
        with open(
            os.path.join(
                GENERATED_CODE_FOLDER,
                f"{MODEL}-{TASK}-{lang1.lower()}-{lang2.lower()}.md",
            ),
            "w",
        ) as fp:
            fp.write(rsp["message"]["content"] + "\n")
        print(f"completed {MODEL} {TASK} {lang1} {lang2}")

completed dolphincoder7b bit_ops C CPP
completed dolphincoder7b bit_ops C GO
completed dolphincoder7b bit_ops C JAVA
completed dolphincoder7b bit_ops C JS
completed dolphincoder7b bit_ops C PY
completed dolphincoder7b bit_ops C RS
completed dolphincoder7b bit_ops CPP C
completed dolphincoder7b bit_ops CPP GO
completed dolphincoder7b bit_ops CPP JAVA
completed dolphincoder7b bit_ops CPP JS
completed dolphincoder7b bit_ops CPP PY
completed dolphincoder7b bit_ops CPP RS
completed dolphincoder7b bit_ops GO C
completed dolphincoder7b bit_ops GO CPP
completed dolphincoder7b bit_ops GO JAVA
completed dolphincoder7b bit_ops GO JS
completed dolphincoder7b bit_ops GO PY
completed dolphincoder7b bit_ops GO RS
completed dolphincoder7b bit_ops JAVA C
completed dolphincoder7b bit_ops JAVA CPP
completed dolphincoder7b bit_ops JAVA GO
completed dolphincoder7b bit_ops JAVA JS
completed dolphincoder7b bit_ops JAVA PY
completed dolphincoder7b bit_ops JAVA RS
completed dolphincoder7b bit_ops JS C
complete